## Work Notebook
### Algorithm of Content-Based Rec Engine - find similar products according a tag classifier followed by cosine similarity by Spacy embeddings of reviewText 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
#loading downloaded data
df=pd.read_json('~/Downloads/reviews_Electronics_5.json', lines=True)

### Loading pickled webscraping files

In [3]:

with open('df.pickle', 'rb') as file:
      
    # Call load method to deserialze
    df = pickle.load(file)


### We will train a model that classifies user input (item) into appropriate tag labels 

In [ ]:
with open('tagzz.pickle', 'rb') as file:
    tagzz = pickle.load(file)

In [ ]:
df

### We will use Spacy Embeddings for the df.reviewText and do cosine similarity to find similar products

In [32]:
from tqdm import tqdm
revs=[]
for x in tqdm(df.asin.unique()[0:]):    
    revs.append(df[df.asin==str(x)].reviewText)

100%|███████████████████████████████████| 63001/63001 [1:22:24<00:00, 12.74it/s]


In [34]:
len(revs)

63001

In [36]:
#cleaning up df
df.drop_duplicates(subset='asin', inplace=True)

In [37]:
df.shape

(63001, 9)

In [52]:
df.reviewText=revs

In [53]:
df.reviewText

0          0    We got this GPS for my husband who is an ...
5          5    I am using this with a Nook HD+. It works...
10         10    This item is just as was described in th...
18         18     This is a great buy, compared to a $60 ...
237        237    The Nook Tablet, in both the 16gb versi...
                                 ...                        
1689106    1689106    The Sabrent CR-CCU3 Card reader is ...
1689125    1689125    Great hub for carrying around if yo...
1689142    1689142    This is an interesting device in th...
1689162    1689162    ...all part of the package that com...
1689183    1689183    Burned these in before listening to...
Name: reviewText, Length: 63001, dtype: object

In [71]:
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
def polar(x):
    data=sid.polarity_scores(x)
    data

In [ ]:
sent=[polar(x) for x in tqdm(revs)]

 93%|███████████████████████████████▍  | 58311/63001 [37:21:50<06:47, 11.51it/s]

In [ ]:
sent[0]

### Then we combine the two inputs 

In [ ]:
model=Models.Sequential()
model1 = Sequential()

model2 = Model(inputs=[model.input, model1.input], outputs=x)